In [1]:
import torch
from src.datamodels.pipeline import DatamodelPipeline
from src.datamodels.config import DatamodelConfig
from src.llms import Llama3_1
from src.evaluator import GleuEvaluator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"	

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
datamodels_path = "../../data/instruction-induction-data/datamodels/proportion_study/210_5"

llama = Llama3_1()

config = DatamodelConfig(
    k = 8,
    num_models= 105,
    datamodels_path = "../../data/instruction-induction-data/datamodels/proportion_study/210_5",
    train_collections_idx = None,
    test_collections_idx = None,
    test_set = None,
    train_set = None,
    instructions= None,
    llm = llama,
    evaluator=GleuEvaluator(),
)


datamodel = DatamodelPipeline(config)



/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]


In [2]:
# datamodel.load_collections_from_path(test_flag=False)

In [3]:
dt = torch.load(f"{datamodels_path}/datasets/train/dt_0.pt")
dt

/tmp/ipykernel_4045853/2093523669.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dt = torch.load(f"{datamodels_path}/datasets/train/dt_0.pt")


In [4]:
train = torch.utils.data.DataLoader(dt, batch_size=100, shuffle=True)
train

In [5]:
len(train)

413

In [6]:
from src.datamodels.models import LinearRegressor

model = LinearRegressor(420, 1)

In [7]:
for i, data in enumerate(train):
    print(data[0].shape)
    print(data[1].shape)
    
    break

torch.Size([100, 420])
torch.Size([100])


In [8]:
pred = model(data[0].to(device)).squeeze(1)
pred[:10]

tensor([-0.0599,  0.0737, -0.0727, -0.0024,  0.0438,  0.0051, -0.0620, -0.0315,
         0.0676, -0.1196], device='cuda:0', grad_fn=<SliceBackward0>)

In [9]:
print(data[1].shape)
print(pred.shape)

torch.Size([100])
torch.Size([100])


In [10]:
import torch.nn as nn
loss = nn.MSELoss()
loss(pred, data[1].to(device)).item()

0.14658334851264954

In [11]:
w = model.get_weights()
w1 = model.get_weights()

In [12]:
w = torch.concat((w, w1), dim=0)
w.shape

torch.Size([2, 420])

In [13]:
w = model.get_bias()
w1 = model.get_bias()

In [14]:
w = torch.concat((w, w1), dim=0)
w.shape

torch.Size([2])

In [15]:
w = torch.tensor([], device=device)
w1 = model.get_weights()
w = torch.concat((w, w1), dim=0)
w.shape

torch.Size([1, 420])

In [16]:
w1.device

device(type='cuda', index=0)

In [17]:
datamodel.train_datamodels(
        epochs=1,
        train_batches=1800,
        val_batches=1000,
        val_size=0.1,
        lr=0.0001,
        random_seed=42,
        patience=5,
        subset=40000,
        device="cuda:0",
)

Idx: 0


/work/caio.rhoden/datamodels-context-reduction/src/datamodels/pipeline/Datamodels.py:216: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(f"{self.datamode

Epoch [1/1], Loss: 0.0860, Val MSE: 0.0920


/home/caio.rhoden/miniconda3/envs/datamodels/lib/python3.11/site-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([1000])) that is different to the input size (torch.Size([1000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [20]:
weights = torch.load(f"{datamodels_path}/estimations/weights.pt")
weights.shape

/tmp/ipykernel_4045853/1083643982.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(f"{datamodels_path}/estimations/weights.pt")


torch.Size([1, 420])

In [21]:
bias = torch.load(f"{datamodels_path}/estimations/bias.pt")
bias.shape

/tmp/ipykernel_4045853/2489783211.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  bias = torch.load(f"{datamodels_path}/estimations/bias.pt")


torch.Size([1])